In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textblob import Word
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import tqdm  # For progress bar

# ---------------------------
# 1. Define Business Models and Categories
# ---------------------------
business_model_categories = {
    "Distributor": [
"Automation",
"Bearing",
"Control Solutions",
"Control System",
"Drives",
"Electrical",
"Electrical & Mechanical",
"Electronics",
"Electronics & Electrical",
'HVAC',
"Hydraulics",
"Industrial Automation",
"Industrial Equipment",
"Industrial Solutions",
"PLC, Drives, Controllers",
"R&D",
"Research & Design",
"Semiconductors"
    ],
    "End-user": [
"Aerospace",
"Agriculture",
"Apparel",
"Appliances",
"Automotive",
"Beverage",
"Chemicals",
"Construction",
"Cranes & Hoist",
"Electric Machinery",
"Electronics",
"Energy",
"Fabricated Metal",
"Food",
"Furniture",
"HVAC",
"Leather",
"Marine",
"Medical",
"Metals",
"Mining",
"Non-metallic",
"Petrol",
"Pharmaceutical",
"Plastics",
"Printing",
"Robotics",
"Rubber",
"Steel",
"Textile",
"Tobacco",
"Transportation Equipment",
"Water",
"Woodal valid categories",
    ],
    "OEM": [
"Agricultural Machinery",
"Apparatus & Machinery for Chemical Industry",
"Automatic Machinery & Equipment",
"Beverage Machinery",
"Boiler",
"Casting",
"Construction Machinery",
"Conveyor",
"Dryers",
"Electric Machinery",
"Equipment for Smelting Plant & Rolling Mills",
"Equipment for Surface Finishing",
"Fishing, Animal & Husbandry Machinery",
"Food Machinery",
"Forging & Stamping",
"Freezing & Air Conditioning Equipment",
"Gaming Machines",
"Grinders, Crushers & Screen Machines",
"Hardware, Metal Products & Other Products of Materials",
"Heat Treatment Equipment",
"Heating, Ventilation, & Air Conditioning (HVAC)",
"Household Machinery",
"Hydraulic Presses, Pneumatic & Hydraulic Machinery",
"Industrial Furnace",
"Industrial Machinery",
"Industrial Turnkey Plant",
"Internal Combustion Engines",
"Laundry & Dry Cleaning Machinery",
"Lifting Equipment",
"Machinery Equipment for Foundry Plant",
"Machinery Equipment for Services",
"Machinery for Clothing Industry",
"Machinery for Leather Industry",
"Machinery for Non-metallic Mineral Products",
"Machinery for Processing Plastics",
"Machinery for Rubber Industry",
"Marine Machinery & Equipment",
"Material Handling Machinery",
"Medical Machines & Equipment",
"Metal Cutting Tools",
"Metal Forming Machinery",
"Mining Machinery",
"NC Machines",
"Office Machinery",
"Packaging Machinery",
"Paper Making & Converting Machinery",
"Pharmaceutical Machinery",
"Piping System",
"Planning, Consulting & Construction of Complete Industrial Plant",
"Pollution Control Equipment",
"Precision Machinery, Inspecting & Testing Equipment",
"Printing & Binding Machinery",
"Productonics Machinery & Equipment",
"Productronics",
"Pumps, Fans, Compressors & Vacuum Pumps",
"Refrigerating Equipment",
"Repair, Maintenance & Installation of Machinery",
"Semiconductor Machinery",
"Sewing Machine",
"Shoe Making Machinery",
"Special Machine Tools",
"Subcontract Services",
"Textile Machinery",
"Tools, Cutters, Moulds, & Dies",
"Transportation Equipment",
"Vending Machine",
"Water / Wastewater Machinery",
"Weighing &  Measuring Machine",
"Welding & Cutting Machinery",
"Woodworking Machinery",
    ],
    "System Integrator": [
"Automation",
"Bearing",
"Control Solutions",
"Control System",
"Drives",
"Electrical",
"Electrical & Mechanical",
"Electronics",
"Electronics & Electrical",
"HVAC",
"Hydraulics",
"Industrial Automation",
"Industrial Equipment",
"Industrial Solutions",
"Motors",
"PLC, Drives, Controllers",
"R&D",
"Research & Design",
"Semiconductors",
    ]
}

# Precompute valid (business model, category) pairs
valid_pairs = {(bm, cat) for bm, cats in business_model_categories.items() for cat in cats}

# Precompute lists of valid business models and categories
valid_b_models = list(business_model_categories.keys())
valid_categories = list({cat for cats in business_model_categories.values() for cat in cats})

# ---------------------------
# 2. Advanced Text Processing Class
# ---------------------------
class TextAnalyzer:
    def __init__(self, valid_pairs, valid_b_models, valid_categories):
        self.valid_pairs = valid_pairs
        self.valid_b_models = valid_b_models
        self.valid_categories = valid_categories
        
        # Build TF-IDF on the combined list of business models and categories
        self.tfidf = TfidfVectorizer(stop_words='english')
        # For semantic matching, we use a dedicated TF-IDF matrix per call
        self.tfidf_matrix = self.tfidf.fit_transform(valid_b_models + valid_categories)
        
    def semantic_match(self, text, valid_options):
        # Instead of matching against the entire matrix, match individually over valid_options
        input_vec = self.tfidf.transform([text])
        best_match = None
        best_score = -1
        for option in valid_options:
            option_vec = self.tfidf.transform([option])
            sim = cosine_similarity(input_vec, option_vec)[0][0]
            if sim > best_score:
                best_score = sim
                best_match = option
        return best_match

    def spell_check(self, text):
        # Using TextBlob's spell check for a quick correction
        return Word(text).correct()
    
    def fuzzy_match(self, text, valid_options):
        # Using fuzzywuzzy to get the closest match
        return process.extractOne(text, valid_options, scorer=fuzz.token_set_ratio)[0]
    
    def validate_pair(self, b_model, category):
        return (b_model, category) in self.valid_pairs

# ---------------------------
# 3. Load Your Main Data
# ---------------------------
df_main = pd.read_csv("Indonesia.csv")

# Initialize analyzer with precomputed valid values
analyzer = TextAnalyzer(valid_pairs, valid_b_models, valid_categories)

# ---------------------------
# 4. Clean "Invalid" Rows
# ---------------------------
def clean_row(row):
    # Convert to lowercase and apply spell checking
    b_model = analyzer.spell_check(row['Business Model'].lower())
    category = analyzer.spell_check(row['Category'].lower())
    
    # If the pair is invalid, try to match with valid options
    if not analyzer.validate_pair(b_model, category):
        b_model = analyzer.semantic_match(b_model, analyzer.valid_b_models)
        category = analyzer.fuzzy_match(category, analyzer.valid_categories)
        
    # Return title-cased values if valid; otherwise, return original values as fallback
    if analyzer.validate_pair(b_model, category):
        return b_model.title(), category.title()
    else:
        return row['Business Model'], row['Category']

# Process only rows flagged as "Invalid"
df_invalid = df_main[df_main['Valid Category/Business Model'] == 'Invalid'].copy()

tqdm.pandas(desc="Processing Invalid Rows")
df_invalid[['Business Model', 'Category']] = df_invalid.progress_apply(clean_row, axis=1, result_type='expand')

# Update the main dataframe with the corrected rows
df_main.update(df_invalid)

# ---------------------------
# 5. Save the Cleaned Data
# ---------------------------
df_main.to_csv("Indonesia_Output.csv", index=False)
print("Data cleaning completed. Output saved to Indonesia_Output.csv.")


Processing Invalid Rows: 100%|██████████| 1161/1161 [55:33<00:00,  2.87s/it] 


Data cleaning completed. Output saved to Indonesia_Output.csv.
